In [2]:
# importing libraries
import pandas as pd
import numpy as np
import gensim
import hm
import pickle
import csv
import matplotlib.pyplot as plt

# import library from gensim  
from gensim.models import CoherenceModel
from gsdmm import MovieGroupProcess

my_doc=pd.read_csv('C:/Users/Kois/lemmatizedCorpus.csv') #Lemmatized tokens
my_doc=my_doc.drop(['Unnamed: 0.1'], axis = 1)
my_doc=my_doc.drop(['Unnamed: 0'], axis = 1)
my_doc=my_doc.to_numpy().tolist()
len(my_doc)
my_doc

new_list=[]
temp_list=[]
for x in my_doc:
    for item in x:
        if str(item) != 'nan':
            temp_list.append(item)
    new_list.append(temp_list)
    temp_list=[]
my_doc=new_list
len(my_doc)

stop_words=['እኔ', 'የእኔ', 'እኔራሤ', 'እኛ', 'የእኛ', 'እኛራሣችን', 'አንቺ', 'ነህ', 'አላችሁ', 'እርሥዎ', 'ትፈልጋለህ', 'ያንተ', 'ራሥህን', 'እራሣችሁ', 'እሡ', 'የእሡ',
 'ራሡ', 'እሷ', 'እሷናት', 'የእሷ', 'እራሷ', 'ነው', 'እነሡ', 'እነሡን', 'የእነሡ', 'ራሣቸው', 'ምንድን', 'የትኛው', 'ማን', 'ይህ', 'የሚልነው', 'ያ', 'እነዚህ', 'እነዚያ', 'ነኝ', 'ናቸው', 'ነበር', 'ነበሩ', 'ሁን', 'ቆይቷል', 'መሆን',
 'አላቸው', 'አለው', 'ነበረው', 'ያለው', 'መሥራት', 'ያደርጋል', 'አደረገ', 'ማድረግ', 'ሀ', 'አንድ', 'የ', 'እና', 'ከሆነ', 'ወይም', 'ምክንያቱም','ነው', 'ነበር',
 'እንደ', 'እሥከ', 'እያለ', 'በ', 'ለ', 'ጋር', 'ሥለ', 'ላይ', 'መካከል', 'ወደ', 'በኩል', 'ወቅት', 'ከዚህበፊት', 'በኋላ', 'ከላይ', 'ከታች', 'ከ', 'ወደላይ', 'ታች', 'ውሥጥ', 'ውጭ', 'በላይ',
 'እንደገና', 'ተጨማሪ', 'ከዚያ', 'አንድጊዜ', 'እዚህ', 'እዚያ', '–','መቼ', 'የት', 'ለምን', 'እንዴት', 'ሁሉም', 'ማንኛውም', 'ሁለቱም', 'እያንዳንዳቸው', 'ጥቂቶች', 'በጣም', 'ሌላ', 'አንዳንድ',
 'እንደዚህ', 'ብቻ', 'የራሡ', 'ተመሣሣይ', 'ሥለዚህ', 'ይልቅ', 'እንዲሁ', 'ት', 'ይችላል', 'ይገባል', 'ይገባኛል', 'አሁን', 'መ', 'ም', 'ኦ', 'ዳግም',
 'መሆን', 'ሁለ', 'ሁለም', 'ህዝብ', 'ሀሙሥ', 'ለመሆኑ', 'ለምንድን', 'ሌሎች', 'መጽሀፍ', 'ማክሠኞ', 'ምን', 'ሠኞ', 'ሠው', 'ሢሆን', 'ሥንት', 'ረቡእ',
 'ቅዳሜ', 'በዚህ', 'ብላ', 'ነገር', 'አለ', 'አርብ', 'አንተ', 'አንዳንድ', 'ኢትዮጵያ', 'እሁድ', 'እናንተ', 'እንኳን', 'እግር', 'ከመሆን', 'ወይንም', 'ዋና', 'ዘንድ', 'የሚከተለው', 'ያኔ', 'ይኼው', 'ገጽ', 'እነርሡ',
 'ን', 'ና', 'ዎች','ኛ', 'ይጠበቃል', 'ብለዋል', 'ሆ', 'ግን','ሁሉ', 'አንቀጽ', 'እንደሆነ','"', 'በማይበልጥ', 'መሠረት', 'ሁኔታ', 'ይሆናል', 'ሆኖ', 'ከአንድ', 'በማናቸውም',
 'ወር', 'ከአምሥት', 'በሆነ', 'ከዚህ', 'የሆነ', 'ሀያ', 'ሆነ', 'በኊላ', 'በአንድ', 'የሆኑ', 'ከአሥራ', 'የሆነውን', 'መሆኑ', 'ሌላውን', 'ከሠባት', 'ለሌላ',
 'አለበት', 'ሢል', 'ይሆናሉ', 'በሙሉ', 'አሥራ', 'ቢሆንም', 'አንዱ', 'የሌላውን', 'ከሁለት','”','የሆኑትን', 'በሆኑ', 'ጀምሮ', 'በመሆን', 'ባለ', 'ይህንን', 'እንዲቆይ', 'ሌላው', 'የሚሆነው', 'በአንዱ', 'ሢባል', 'ሣለ', 'የሆነው', 'መሆናቸው', 'በዋና', 'በማቀድ', 'ጊዜና', 'ለዚህ', 'ሦሥተኛ', 'የነገሩ', 'ሥድሥት', 'በሆነው', 'ይሁን', 'ከዚሁ', 'በእነዚህ', 'ከማናቸውም', 'ከነበረው',
 'በአንዳንድ', 'በእያንዳንዱ', 'ጊዜም', 'አሥከ', 'የሌሎች', 'የሚሆኑት', 'ከሆነው', 'የነበረውን', 'ያሉ', 'ከሌሎች', 'አንዲት', 'ለሌሎች', 'ለሆነው', 'ሠኣት', 'ብሎ', 'ከሠላሣ',
 'የሚሆኑ', 'ላይም', 'የሆናል', 'ከነዚህ', 'ያህል', 'ከሆነና', 'ለሆኑት', 'እነዚሁ', 'እንደሆኑ', 'ሥለማናቸውም', 'ሥለዚሁ', 'ከአንዳንድ',
 'በእነዚሁ', 'በአምሥት', 'የሆኑበታል', 'ለነዚህ', 'ለማንኛውም', 'አንደኛ', 'ይኸኛው', 'ከርሡ', '“','መሆኑን', 'ለዚያው', 'ለዚሁ', 'ለእነርሡም', 'እዚሁ', 'ሀ', 'ረ', 'ሸ', 'አምሥት',
 'ከሦሥት', 'በተለይም', 'በሌላ', 'ሺህ', 'ማናቸውንም', 'ከአሥር', 'የማይበልጥ', 'እንዲሁም', 'ይህን', 'የዚህ', 'ማናቸውም', 'ከሥድሥት', 'መቶ', 'ያለ', 'አንድን',
 'ያላቸውን', 'ሊሆን', 'ሦሥት', 'ካልሆነ', 'ቢያንሥ', 'ቢሆን', 'እነዚህን', 'አንዱን', 'ሁለት', 'ወይዘሮ', 'ተብሎ', 'ሣይሆን', 'እንደሆነና', 'ከብር', 'ሆኖም', 'የነበሩ',
 'የሌላ', 'ያላቸው', 'ይህንኑ', 'ሆነው', 'በሥተቀር', 'ሥም', 'እንደገና', 'የማያንሥ', 'እጅግ', 'እንዲሆን', 'እንኳ', 'ከሀያ', 'ከሀምሣ', 'ይኸው', 'ለአንድ', 'የሚችለውን',
 'በሚገባ', 'ይህም', 'እንዲሆኑ', 'ከሌላ', 'ለሆነ', 'በሌሎች', 'አንደሆነ', 'እንዲህ', 'በነዚሁ', 'በእንደዚህ', 'ሥምንት', 'ሢሆንና', 'ምንጊዜም', 'ለማናቸውም', 'የአንድ', 'እነዚህኑ', 'ሢሆኑ', 'በሁለቱም',
 'እንደነዚህ', 'የሆኑት', 'የማናቸውም', 'ይህንንም', 'የአንድን', 'በሙሉም', 'በነዚህ', 'የዚሁ', 'ለእያንዳንዱ', 'ሥለሆነ', 'መሆናቸውን', 'ማንኛውንም', 'ሁለቱ', 'እንጂ',
 'ከሥምንት', 'ሁለቱንም', 'በሁለት', 'በእሥር', 'በሚል', 'ቁጥር', 'ባሉ', 'ከመቶ', 'እነዚህም', 'ሢኖር', 'ሠላሣ', 'ለሆኑ', 'ሠባት', 'እነደሆነ', 'ይህችው', 'ከእነዚህ',
 'ከእነዚሁ', 'የአንቀጹ', 'ወይ', 'የሆነችን','እኮ','ኧረ','ጋ','እንዴ','ነበረ','ነዉ','ነሀ','ናቸዉ']


def stopWordRemoval(tokens):
    token_new=[]
    for x in tokens:
        if x not in stop_words:
            token_new.append(x)
    return token_new

clean_doc=[]
for x in my_doc:
    clean_doc.append(stopWordRemoval(x))
#     print(stopWordRemoval(x))

#remove verbs
listOfVerbs="ተሰለፈ,ሰጠ,ገባ,አወቀ,መሰለ,መጣ,ጠማ,ነደደ,ወጣ,ለየ,ጣለ,ነዳ,ኖረ,ተከለ,ፈለገ,ወሰነ,ቻለ,ተገኛኘ,መታ,ሰረረ,ባከነ,አነሰ,ደገመ,ያዘ,አነሳ,አለቀ,አስቀመጠ,ለበሰ,ላከ,አወገዘ,አካሄደ,ገደለ,አለፈ,ቀረረ,ለቀቀ,ለመጠ,ጀመረ,ተናገረ,ገመተ,ዶለ,ፈጠረ,ሰቀቀ,ፈጸመ,ቀረ,አየ,ቈየ,ዘገበ,በላ,ተባለ,ፈላ,ቀነተ,ፈታ,ሞተ,ገለጸ,ገለጠ,ደበነ,ሰደደ,ጐለተ,ተቈጣ,ማረ,አፈረ,ዘረዘረ,ጠበቀ,ተወ,መረጠ,ተቀበለ,ተመቸ,ደረደረ,ተመለከተ,ጨፈነ,ዞረ,ዠለጠ,ነፈሰ,ተሸነፈ,አማ,ረጋ,ተቃወመ,ቀየረ,አለመ,ገገረ,ቀለለ,ተዘጋጀ,ዋለ,ቆመ,ተላለፈ,ቀነበረ,ጨረሰ,ቋጠረ,ተሰራጨ,አራ,ሾመ,ስምር,ተራመደ,ቀፈፈ,ተቋቋመ,ረገበ,ሰከረ,ለቀለቀ,ታገለ,አጐነበሰ,አሰረ,ረገጠ,ተጠረጠረ,ነካ,ተነሳ,አበለ,ጠቀመ,አረገ,ጠፋ,ወደቀ,ነዛ,ደረሰመ,ገረረ,ቀጠፈ,ረሳ,ፈጀ,ሰለቸ,ለወጠ,ደከመ,ሄደ,ሰደበ,ቀላ,ቀረጸ,ፎከረ,ወቀረ,ተማረ,ወለደ,ሳበ,ሰራ,ቀደሰ,አደረ,ለመደ,ተረጐመ,ሰፈነ,ፈነጠቀ,ቀለደ,ገበየ,ወቃ,አመሰ,አጣ,ዘፈነ,ተደረገ,ወሰደ,ነጐደ,ቀቀለ,አናገረ,በሳ,መጠነ,ተካሄደ,ተንጠባጠበ,ረዘመ,መከረ,መሰከረ,ከለከለ,ፈራ,ወደደ,ደገፈ,ተጫወተ,ለመለመ,ተሰቃየ,ዘና,ተሸበረ,ገነባ,ጨበጨበ,ቈረቈረ,ቈረጠ,ጨቈነ,ቀረበ,ተሳበ,ሸጠ,ገዛ,ላላ,ባሰ,ተቀመጠ,ቀጠለ,ደፈነ,ተመለከ,ቀደመ,ፈሳ,ተመኘ,ነቃ,መጠጠ,ሮጠ,ተመካ,ቀደደ,ተከተለ,ተወራ,ናደ,ቀጣ,ገታ,ቀሰቀሰ,ተሻገረ,ጠየቀ,ፋቀ,ጣረ,ተደሰተ,ለመነ,ወተወተ,ቀማ,በጠበጠ,ዋጠ,ተከበረ,ቸገረ,ተቀየመ,ተጠቃ,አረመ,ዘረፈ,ጠራ,ሳቀ,አስተማረ,ጠረገ,አመጠ,ተያያዘ,ጻፈ,ገረመ,አከከ,ደረቀ,ተንበረከከ,ተለጐመ,ከሳ,ተጨነቀ,ዋሸ,ተታለለ,ሰፈረ,ፈተነ,በረከተ,ተከላከለ,ተሳተፈ,ገፋ,ተመሰገነ,ሰለጠነ,ተንቀሳቀሰ,ተወዳጀ,ጨፈረ,ተመጻደቀ,ተባበረ,ጠነከረ,ተቀዳጀ,ተንፈረጠጠ,ጠቀሰ,ቈጠረ,ዘረጋ,ጠቀለለ,ባረከ,ሳተ,ሸረሞጠ,ላጠ,ዳጠ,ነጠቀ,ተከናወነ,ላገ,በደለ,ጨበጠ,ዘጋ,ጨቀጨቀ,ላፈ,ነሳ,ወረረ,ዳነ,ዘከዘከ,አቀደ,ተገለገለ,ተመነ,አመመ,ገጠመ,ተመረኰዘ,ተዳረሰ,ታጠቀ,አፈነ,ተንከባከበ,ኰነነ,ተቃጠለ,አረደ,ፈረሰ,ተደራጀ,ጠለዘ,ቀረጠፈ,ተዘናጋ,ሻ,ከሰሰ,ረገመ,ተገደደ,ተጨናነቀ,ጠረቀመ,ቀነሰ,ረካ,ዳበረ,ጐለበተ,አለበ,ወረወረ,ጨፈለቀ,ተኛ,ደበቀ,ቈረጠመ,መዘገበ,ላመ,ወከለ,ነቀለ,ኰሳ,ሸለለ,አሸ,ጠዳ,ተተራመሰ,ዘመተ,ጸደቀ,ከተመ,ሰካ,ታወረ,ተሳካ,ነጣ,አገዘ,ገደበ,ተሳሳተ,ለፋ,ተበላሸ,ሸመቀ,ናፈቀ,ሰበሰበ,ጠለፈ,ተከፋ,ተሻሻለ,ከሰመ,ፈቀደ,ተጋ,ጐደለ,ቈጨ,ነባ,አስቻለ,በረረ,ከፈተ,አለለ,ጠመመ,ወጠረ,ገበረ,ታታ,ገለበጠ,ሸመገለ,ደነበረ,ተደናቀፈ,ሰላ,መረረ,ጠቈመ,ጋበዘ,ቀበረ,ጫነ,ናቀ,ሰወረ,ቀሰረ,ተቸ,መረመረ,መዘነ,ወደመ,ተወዳደረ,ተከባበረ,ተዳደረ,ተሰኘ,ጋረጠ,ፈሰሰ,ገነዘ,አደለ,አሸገ,ደፋ,ጸለየ,ደወለ,ጠጣ,ተጠጋ,ዳመጠ,መሰገ,ቀላቀለ,ማለ,ዘመረ,ሰየመ,ጣሰ,ለቀመ,አከፋፈለ,ጸነነ,በጠሰ,ጣፈ,ታወሰ,አለቀሰ,አበደ,ዘራ,አጨደ,ተጨማለቀ,ጮኸ,ከሰከሰ,መሸ,ታበየ,ጋረ,ማረረ,አጽናና,ተግባባ,መነጨ,አጠበ,ረጨ,ለፈለፈ,አደሰ,ሞገተ,አቻለ,ወረሰ,ሞከረ,ተሳነ,ወፈረ,ተከዘ,ነከሰ,አረረ,ተዛዘበ,ሸተተ,ፈነቀለ,ወቀሰ,ሳሳ,ዘነጋ,ገራ,ተበጀ,ካሰ,ደነዘዘ,ረበረበ,ጸዳ,ተከታተለ,ተገነዘበ,አዘነበለ,ተወሀደ,ራቀ,ዘነነ,ገጨ,ከተበ,ሸረሸረ,አለቃቀሰ,ተነገረ,ተላላከ,ቀዳ,ጠባ,ፈነዳ,ተወናበደ,ተስተካከለ,ጠበበ,ቈሸሸ,ተጠነቀቀ,ደነገጠ,ተጓዘ,አቀፈ,መገበ,ፈካ,ፈወሰ,ሻረ,ዛለ,ተከራየ,አስተዋለ,ጠበሰ,ተኰሰ,ተጠና,ተነፈሰ,በተነ,ቈፈረ,ጠረበ,ተደመመ,ዘቀጠ,ተዋከበ,ቃመ,ናወዘ,ተልከሰከሰ,አርጋጋ,ተሽከረከረ,ተለተመ,ተጣደፈ,ዋተተ,ተገለለ,ዶለተ,ነፈገ,ተጠናቀቀ,በሰለ,ደመደመ,አከተ,ሰጋ,ማረከ,በረቀ,ቈለለ,ማቀቀ,ተወገደ,ተኰላሸ,ዋጀ,ተረተ,ተስማማ,ረባ,ፈጠጠ,ፋረ,ሰነጠቀ,ደበደበ,ደፈረሰ,ተጠናወተ,ቀሰመ,ኰላ,አጠፈ,ጣመ,አፈሰ,ነፋ,ረፈደ,ተዋረደ,ሰነዘረ,ተቋረጠ,አወጋ,ተንጠለጠለ,አረሰ,ከለበ,ተከራከረ,ፈተሸ,አተመ,ቈሰለ,ተደለደለ,ገመገመ,ተነጻጸረ,ተዋወቀ,ተሸከመ,አረቀ,አገደ,ተቻቻለ,ሰመጠ,ወጋ,ተደመጠ,ፈለፈለ,ጨከነ,ከተተ,ዘባረቀ,መረተ,አመለጠ,ነከተ,ተመቻቸ,ሸፈነ,ጠመዘዘ,ሰበረ,ሰረቀ,ወለቀ,አነበበ,ገረፈ,መረቀ,አጠረ,ተምታታ,ሰበከ,ዳረ,ሰዋ,ለኰሰ,ተካረረ,አደነ,ራሰ,ቀረፈ,ሰለመ,ጠነባ,ሰገደ,ጐበኘ,ከረፋ,አኘከ,ወነጀለ,ተጣመረ,ጓጓ,ዘነጠ,ደበለቀ,ፈዘዘ,አጀበ,ከተፈ,መደበ,ቀጠረ,ተለማመጠ,ተበደረ,ጠፈፈ,ታገሰ,አነቀ,ሰነበተ,ደረተ,ሰቀለ,ደፈረ,ዘለፈ,ጨፈጨፈ,ረቀቀ,ተነጋገረ,ተከፈፈ,ተሰናበተ,ላሰ,ሰለበ,ዳሰሰ,ራበ,ሸኘ,ቀባጠረ,ተርበተበተ,አመሸ,ተሳለቀ,ቸከ,ገዘገዘ,ዳከረ,ገዘፈ,ጐተተ,ጋጠ,ጾመ,ወቀጠ,ተሞገሰ,ጠገበ,ከጀለ,ተሸቀበ,ረሸነ,ተረከበ,ዘገነነ,ከረከረ,ተሞሸረ,ፈረመ,ሸመተ,ከበተ,አስመለሰ,ፈለጠ,ረበሸ,ደጐመ,ተነጣጠረ,ሰረሰረ,ገላገለ,ተዛመደ,ተጃጃለ,በሰበሰ,ዋለለ,ተበሳጨ,ረጠበ,ተንጨረጨረ,ተረከ,ኰለኰለ,ካነ,አረጠ,ካደ,ወዘወዘ,ረበበ,ተቈጣጠረ,ተጸጸተ,አላመጠ,አካፈለ,ሸመጠጠ,አኰረፈ,ለጠፈ,አወከ,ገነጠለ,ጐነጐነ,ቀለመ,ተለመ,ቈጠበ,ዛገ,ነደለ,ገመደ,ሰገሰገ,ጋረደ,ተጠተተ,መረዘ,ደረገመ,ሸና,ተጸነሰ,ጨነገፈ,አስተናገደ,ሸቀጠ,ነቀነቀ,ተቃረነ,ተመለጠ,ተፋ,አሾፈ,ሸሸ,ጠረዘ,ተበላለጠ,ቀባ,ደለበ,ዋኘ,ፈረጠ,ከለሰ,በረደ,ተሸማቀቀ,ቀጠቀጠ,ሰነገ,ተግደረደረ,ቀየሰ,ጫረ,አነጠሰ,ተዘዋወረ,ተካፈለ,ተጥለቀለቀ,ተጭበረበረ,ሰከነ,ተዘወረ,ተንቀጠቀጠ,ጠመጠመ,ሰረዘ,አስተጋባ,ከሰረ,ጠለቀ,ተንከረፈፈ,ዘመመ,ሞቀ,ዳሰ,ፈነጠረ,ተለቀ,ቀበጠ,ተገናዘበ,ከሸፈ,ተነተነ,ዳረገ,ቃኘ,ገፈፈ,ጃጀ,ሰቀጠጠ,ተፍጨረጨረ,ነደፈ,ጋለ,ተንሳሳ,ደገሰ,ጀገነ,ቀለጠ,ዛተ,መለለ,ተባባለ,ተስፈነጠረ,ጠመደ,አነከሰ,ጠመቀ,ተገመሰ,ለጠቀ,ማገ,ቋጨ,ተዋሰ,ገሰገሰ,ወረፈ,ተቀኘ,ተነኰተ,ቀላወጠ,ሰየፈ,ጨለፈ,ቸረ,ተረተረ,በከለ,ተወላከፈ,አመላከተ,ከለለ,አጐረ,መነመነ,አበሰ,ደገነ,ሰነፈ,ተጠየፈ,ተነገተ,ወገረ,ቻቻለ,ቀጨጨ,ከረከመ,ራደ,ተኰፈሰ,ተነጠለ,ታጨ,ተንጫጫ,ወሰለተ,ወሸቀ,ኰሰመነ,ተነጫነጨ,ዘረገፈ,ተንሳፈፈ,ተፋጨ,ሾለ,ተንፈራፈረ,ጐረበጠ,ጠቈረ,ቀመሰ,ነተበ,ደላ,ፈረጠጠ,ደነቈረ,መነጠረ,ደመሰሰ,ተዛወረ,ሳገ,ተባ,ሻጠ,ሳመ,ኰረጀ,ረከሰ,ቀዘነ,ከሸነ,በዘበዘ,ተፋለመ,ተደለቀ,ተፈናጠጠ,ከካ,ወዛ,ፈነጨ,አጠነ,ዳበሰ,አመራ,ረገፈ,ገለበ,ፈየደ,ተራቈተ,ጐሰቈለ,ሸቀለ,ሰበቀ,ተቅነዘነዘ,ሰለቀ,ለዘበ,ደሰመ,አሴረ,ተጋዘ,ለሰለሰ,ቀሰፈ,ተንከራተተ,ማገደ,ፈነከተ,ዘላበደ,ዘበዘበ,ተንደላቀቀ,ተደረሰ,ሸረፈ,ተቅለሰለሰ,ተላከከ,ተሻማ,ደፈቀ,ነጠረ,ዘረረ,ደለለ,ተቀጣጠለ,ተውተረተረ,ጐረፈ,ተደረበ,ቸለሰ,ተንደረደረ,ተናጠ,ተከማቸ,ፈተፈተ,ተንሸራተተ,ቀጠነ,ተትረፈረፈ,ኳለ,ጨለመ,ነፈረ,ከሰለ,ተፋጀ,ተዳበለ,ወጠነ,አጋመሰ,ለበለበ,ሸመቀቀ,ተካተተ,ተዋበ,አከመ,ኰረፈ,በገነ,ደበዘዘ,ጠበጠበ,ተቃቃመ,ጋመ,መነዘረ,ዘየደ,ወረዛ,ቀነጠሰ,ነቀሰ,ተገዘተ,ተረባረበ,ጐበዘ,ለጠጠ,ደደበ,ተንጣለለ,ተንቀለቀለ,ሰጠመ,ታከከ,ተቈራ,ጠገነ,ቦረቀ,ከተከተ,ተሽኰረመመ,አመለከተ,ተቀለበ,ለሰነ,ተቈራኘ,ሞገደ,ፈነደቀ,መሰነ,ተዝረከረከ,ሸመደደ,ተንበለበለ,ዘበተ,ቦዘ,ገነፈለ,ነቀፈ,ተጸየፈ,ተደላደለ,ደሰሰ,ነዘረ,ሳለ,ቀዘቀዘ,ነዘነዘ,ደደረ,ተበጠረ,ባተ,ተመሳቀለ,አጠራጠረ,ገነገነ,ነከረ,ተሳደደ,አለጠ,ደቀደቀ,መተረ"
listOfVerbs=listOfVerbs.split(",")
no_verb=[]
verb_set=set(listOfVerbs)

for item in clean_doc:
    item_set=set(item)
    no_verb.append(list(item_set.difference(verb_set)))

clean_doc=no_verb #experiment one, with verbs #experiment two, without verbs

# cast tweets to numpy array
docs = clean_doc
#docs = processed_lemma
# create dictionary of all words in all documents
# initialize a Dictionary
dictionary = gensim.corpora.Dictionary(docs)

# filter extreme cases out of dictionary
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

# create variable containing length of dictionary/vocab
vocab_length = len(dictionary)

# create BOW dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

# define function to get words in topics
def get_topics_lists(model, top_clusters, n_words):
    '''
    Gets lists of words in topics as a list of lists.
    
    model: gsdmm instance
    top_clusters:  numpy array containing indices of top_clusters
    n_words: top n number of words to include
    
    '''
    # create empty list to contain topics
    topics = []
    
    # iterate over top n clusters
    for cluster in top_clusters:
        #create sorted dictionary of word distributions
        sorted_dict = sorted(model.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:n_words]
        if len(sorted_dict)!=0:
            #create empty list to contain words
            topic = []

            #iterate over top n words in topic
            for k,v in sorted_dict:
                #append words to topic list
                topic.append(k)

            #append topics to topics list    
            topics.append(topic)
    
    return topics

result = {}
f_everything=open("everythingGibbs.txt","a")
for k_ in range(2,11):
    print("number of topic=",k_)
    for alpha_ in range(0, 11, 1):
        for beta_ in range(0,11, 1):
            gsdmm = MovieGroupProcess(K=k_, alpha=alpha_/10, beta=beta_/10, n_iters=9)
            gsdmm.fit(docs, vocab_length)
            # print number of documents per topic
            doc_count = np.array(gsdmm.cluster_doc_count)
            # print('Number of documents per topic :', doc_count)
            
            # Topics sorted by the number of document they are allocated to
#             print(doc_count)
            top_index = doc_count.argsort()[-15:][::-1]
            # get topics to feed to coherence model
            topics = get_topics_lists(gsdmm, top_index, 50) 

            # evaluate model using Topic Coherence score
            cm_gsdmm = CoherenceModel(topics=topics, 
                                      model=gsdmm,
                                      dictionary=dictionary, 
                                      corpus=bow_corpus, 
                                      texts=docs, 
                                      coherence='c_v')

            # get coherence value
            coherence_gsdmm = cm_gsdmm.get_coherence()  
            
            #save model
            filename = "gsdmm_k{}_a{}_b{}_c{}.pkl".format(k_,alpha_,beta_,coherence_gsdmm)
            pickle.dump(gsdmm, open(filename, 'wb'))
            
            if coherence_gsdmm >= 0.4:
                print((k_,alpha_,beta_))
                result[(k_, alpha_,beta_)] = coherence_gsdmm
                f=open("hyperparameters_gibbs.txt","a") #check file in case loop terminates unexpectedly
                f.write(str((k_,alpha_/10,beta_/10,coherence_gsdmm)))
                f.close()
                
            f_everything.write(str((k_,alpha_/10,beta_/10,coherence_gsdmm)))

f_everything.close()
#print(result)

with open('testGibbs.csv', 'w') as f:
    for key in result.keys():
        f.write("%s, %s\n" % (key, result[key]))


number of topic= 2
In stage 0: transferred 3490 clusters with 2 clusters populated
In stage 1: transferred 688 clusters with 2 clusters populated
In stage 2: transferred 141 clusters with 2 clusters populated
In stage 3: transferred 34 clusters with 2 clusters populated
In stage 4: transferred 12 clusters with 2 clusters populated
In stage 5: transferred 8 clusters with 2 clusters populated
In stage 6: transferred 8 clusters with 2 clusters populated
In stage 7: transferred 6 clusters with 2 clusters populated
In stage 8: transferred 8 clusters with 2 clusters populated
In stage 0: transferred 2951 clusters with 2 clusters populated
In stage 1: transferred 1746 clusters with 2 clusters populated
In stage 2: transferred 1432 clusters with 2 clusters populated
In stage 3: transferred 1296 clusters with 2 clusters populated
In stage 4: transferred 1163 clusters with 2 clusters populated
In stage 5: transferred 1228 clusters with 2 clusters populated
In stage 6: transferred 1170 clusters w

In stage 3: transferred 1599 clusters with 2 clusters populated
In stage 4: transferred 1510 clusters with 2 clusters populated
In stage 5: transferred 1513 clusters with 2 clusters populated
In stage 6: transferred 1368 clusters with 2 clusters populated
In stage 7: transferred 1304 clusters with 2 clusters populated
In stage 8: transferred 1292 clusters with 2 clusters populated
In stage 0: transferred 3070 clusters with 2 clusters populated
In stage 1: transferred 2000 clusters with 2 clusters populated
In stage 2: transferred 1728 clusters with 2 clusters populated
In stage 3: transferred 1579 clusters with 2 clusters populated
In stage 4: transferred 1467 clusters with 2 clusters populated
In stage 5: transferred 1346 clusters with 2 clusters populated
In stage 6: transferred 1223 clusters with 2 clusters populated
In stage 7: transferred 1096 clusters with 2 clusters populated
In stage 8: transferred 1007 clusters with 2 clusters populated
In stage 0: transferred 2984 clusters wi

In stage 6: transferred 1122 clusters with 2 clusters populated
In stage 7: transferred 1160 clusters with 2 clusters populated
In stage 8: transferred 1039 clusters with 2 clusters populated
In stage 0: transferred 3155 clusters with 2 clusters populated
In stage 1: transferred 2336 clusters with 2 clusters populated
In stage 2: transferred 1923 clusters with 2 clusters populated
In stage 3: transferred 1588 clusters with 2 clusters populated
In stage 4: transferred 1306 clusters with 2 clusters populated
In stage 5: transferred 1151 clusters with 2 clusters populated
In stage 6: transferred 1065 clusters with 2 clusters populated
In stage 7: transferred 1073 clusters with 2 clusters populated
In stage 8: transferred 1058 clusters with 2 clusters populated
In stage 0: transferred 3176 clusters with 2 clusters populated
In stage 1: transferred 2303 clusters with 2 clusters populated
In stage 2: transferred 1940 clusters with 2 clusters populated
In stage 3: transferred 1757 clusters wi

In stage 0: transferred 3257 clusters with 2 clusters populated
In stage 1: transferred 2430 clusters with 2 clusters populated
In stage 2: transferred 2018 clusters with 2 clusters populated
In stage 3: transferred 1603 clusters with 2 clusters populated
In stage 4: transferred 1369 clusters with 2 clusters populated
In stage 5: transferred 1298 clusters with 2 clusters populated
In stage 6: transferred 1227 clusters with 2 clusters populated
In stage 7: transferred 1235 clusters with 2 clusters populated
In stage 8: transferred 1149 clusters with 2 clusters populated
In stage 0: transferred 3483 clusters with 2 clusters populated
In stage 1: transferred 690 clusters with 2 clusters populated
In stage 2: transferred 148 clusters with 2 clusters populated
In stage 3: transferred 28 clusters with 2 clusters populated
In stage 4: transferred 8 clusters with 2 clusters populated
In stage 5: transferred 5 clusters with 2 clusters populated
In stage 6: transferred 4 clusters with 2 clusters

In stage 3: transferred 1421 clusters with 2 clusters populated
In stage 4: transferred 1394 clusters with 2 clusters populated
In stage 5: transferred 1242 clusters with 2 clusters populated
In stage 6: transferred 1171 clusters with 2 clusters populated
In stage 7: transferred 1030 clusters with 2 clusters populated
In stage 8: transferred 987 clusters with 2 clusters populated
In stage 0: transferred 3063 clusters with 2 clusters populated
In stage 1: transferred 1957 clusters with 2 clusters populated
In stage 2: transferred 1657 clusters with 2 clusters populated
In stage 3: transferred 1545 clusters with 2 clusters populated
In stage 4: transferred 1518 clusters with 2 clusters populated
In stage 5: transferred 1523 clusters with 2 clusters populated
In stage 6: transferred 1373 clusters with 2 clusters populated
In stage 7: transferred 1218 clusters with 2 clusters populated
In stage 8: transferred 1174 clusters with 2 clusters populated
In stage 0: transferred 3063 clusters wit

In stage 6: transferred 1163 clusters with 2 clusters populated
In stage 7: transferred 1083 clusters with 2 clusters populated
In stage 8: transferred 1058 clusters with 2 clusters populated
In stage 0: transferred 3120 clusters with 2 clusters populated
In stage 1: transferred 2160 clusters with 2 clusters populated
In stage 2: transferred 1853 clusters with 2 clusters populated
In stage 3: transferred 1667 clusters with 2 clusters populated
In stage 4: transferred 1386 clusters with 2 clusters populated
In stage 5: transferred 1205 clusters with 2 clusters populated
In stage 6: transferred 1139 clusters with 2 clusters populated
In stage 7: transferred 1080 clusters with 2 clusters populated
In stage 8: transferred 1028 clusters with 2 clusters populated
In stage 0: transferred 3108 clusters with 2 clusters populated
In stage 1: transferred 2255 clusters with 2 clusters populated
In stage 2: transferred 1776 clusters with 2 clusters populated
In stage 3: transferred 1490 clusters wi

In stage 0: transferred 3134 clusters with 2 clusters populated
In stage 1: transferred 2295 clusters with 2 clusters populated
In stage 2: transferred 1883 clusters with 2 clusters populated
In stage 3: transferred 1462 clusters with 2 clusters populated
In stage 4: transferred 1266 clusters with 2 clusters populated
In stage 5: transferred 1286 clusters with 2 clusters populated
In stage 6: transferred 1208 clusters with 2 clusters populated
In stage 7: transferred 1172 clusters with 2 clusters populated
In stage 8: transferred 1167 clusters with 2 clusters populated
In stage 0: transferred 3162 clusters with 2 clusters populated
In stage 1: transferred 2374 clusters with 2 clusters populated
In stage 2: transferred 2047 clusters with 2 clusters populated
In stage 3: transferred 1771 clusters with 2 clusters populated
In stage 4: transferred 1494 clusters with 2 clusters populated
In stage 5: transferred 1400 clusters with 2 clusters populated
In stage 6: transferred 1299 clusters wi

KeyboardInterrupt: 

import pandas as pd
import numpy as np
import gensim
import hm
import matplotlib.pyplot as plt

x=pd.read_csv('C:/Users/Kois/Downloads/test7_header.csv')

In [ ]:
#x.loc[(x['K']==2)]

k_values=x['K']
coherences=x['coherence']

plt.plot(k_values, coherences)
plt.xlabel("Number of Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()